**Set environment**

In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [2]:
FD_REG=${FD_RES}/results/region/annotation_ccres
FN_REG=ccres_v4.bed.gz

echo +++++++++++++++++++++++++++++++++++
ls -lh ${FD_REG}/${FN_REG}
fun_head ${FD_REG}/${FN_REG} | wc -l
echo

echo +++++++++++++++++++++++++++++++++++
fun_head ${FD_REG}/${FN_REG}
echo

#echo +++++++++++++++++++++++++++++++++++
#tail ${FD_REG}/${FN_REG}
#echo

+++++++++++++++++++++++++++++++++++
-rw-rw-r-- 1 kk319 reddylab 32M Jun 11  2023 /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_ccres/ccres_v4.bed.gz

gzip: stdout: Broken pipe
10

+++++++++++++++++++++++++++++++++++
chr1	10033	10250	EH38E2776516	0	.	10033	10250	225,225,225	Low-DNase	All-data/Full-classification
chr1	10385	10713	EH38E2776517	0	.	10385	10713	225,225,225	Low-DNase	All-data/Full-classification
chr1	16097	16381	EH38E3951272	0	.	16097	16381	225,225,225	Low-DNase	All-data/Full-classification
chr1	17343	17642	EH38E3951273	0	.	17343	17642	225,225,225	Low-DNase	All-data/Full-classification
chr1	29320	29517	EH38E3951274	0	.	29320	29517	225,225,225	Low-DNase	All-data/Full-classification
chr1	66350	66509	EH38E3951275	0	.	66350	66509	225,225,225	Low-DNase	All-data/Full-classification
chr1	78990	79238	EH38E3951276	0	.	78990	79238	225,225,225	Low-DNase	All-data/Full-classification
chr1	79304	79545	EH38E2776519	0	.	79304	79545	225,225,225	Low-DNase	All-da

In [3]:
ASSAY=KS91_K562_ASTARRseq
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count

echo ${FD_INP}
ls ${FD_INP}/*WGS*.bed.gz | xargs -n 1 basename

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count
KS91_K562_ASTARRseq.Input.rep1.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep2.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep3.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep4.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep5.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep6.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep1.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep2.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep3.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep4.WGS.unstranded.bed.gz


In [4]:
FP_INPS=($(ls ${FD_INP}/*WGS*.bed.gz))
echo Total: ${#FP_INPS[@]}

Total: 10


**RUN**

In [10]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/coverage_ccres_astarr.%a.txt \
    --array 0-9 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=KS91_K562_ASTARRseq

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output directory
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FD_OUT=${FD_RES}/results/${ASSAY}/coverage_ccres_v4

FP_INPS=($(ls ${FD_INP}/*WGS*.bed.gz))
FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})

FN_OUT=${FN_INP}
FP_OUT=${FD_OUT}/${FN_OUT}

FD_REG=${FD_RES}/results/region/annotation_ccres
FN_REG=ccres_v4.bed.gz
FP_REG=${FD_REG}/${FN_REG}

### Show I/O file
echo "Input  file: " ${FP_INP}
echo
echo "Show the first few lines of the input file"
fun_head ${FP_INP} | head -n 5 
echo

echo "Input  file: " ${FP_REG}
echo
echo "Show the first few lines of the input file"
fun_head ${FP_REG} | head -n 3 
echo

### execute
bedtools map \
    -a ${FP_REG} \
    -b ${FP_INP} \
    -o sum \
    -f 0.5 \
    -F 0.5 \
    -e \
| gzip -c \
> ${FP_OUT}

### Show I/O file
echo
echo "Output file: " ${FP_OUT}
echo
echo "Show the first few lines of the output file"
fun_head ${FP_OUT}
echo

### print end message
timer_stop=`date +%s`
runtime=$(echo "${timer_stop} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 30632409


**Check results**

In [12]:
cat ${FD_LOG}/coverage_ccres_astarr.0.txt

Hostname:           x1-02-2.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         01-18-24+14:57:38

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Input.rep1.WGS.unstranded.bed.gz

Show the first few lines of the input file
chr1	10001	10143	chr1_10001_10143	1	.
chr1	10002	10118	chr1_10002_10118	1	.
chr1	10002	10131	chr1_10002_10131	1	.
chr1	10002	10203	chr1_10002_10203	1	.
chr1	10002	10209	chr1_10002_10209	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_ccres/ccres_v4.bed.gz

Show the first few lines of the input file
chr1	10033	10250	EH38E2776516	0	.	10033	10250	225,225,225	Low-DNase	All-data/Full-classification
chr1	10385	10713	EH38E2776517	0	.	10385	10713	225,225,225	Low-DNase	All-data/Full-classification
chr1	16097	16381	EH38E3951272	0	.	16097	16381	225,225,225	Low-DNase	All-data/Full-classification


Output file:  /data/reddylab/Kuei/out/proj_combef